### customized loss function for keras

In [75]:
from keras.preprocessing import image
import keras.backend as K
from keras.losses import binary_crossentropy

In [103]:
def jaccard(y_true, y_pred):
    smooth = 1e-15
    intersection = K.sum(y_true * y_pred, axis=[-1,1,2])
    union = K.sum(y_true + y_pred, axis=[-1,1,2])
    jac = (intersection + smooth) / (union - intersection + smooth)
    return K.mean(jac)

def dice(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    return (2 * intersection + 1e-15) / (union + 1e-15)

def NLL(y_true, y_pred):
    H = K.mean(K.binary_crossentropy(y_true, y_pred), axis = [-1,1,2])
    return K.mean(H)

def customLoss(y_true, y_pred):
    H = NLL(y_true, y_pred)
    J = jaccard(y_true, y_pred)
    return (H-K.log(J))

In [6]:
#### load ytrue
mask = image.load_img("../data/sample_mask/mask.jpg", target_size=(224, 224)) # shape(244,244,3)
y_true = image.img_to_array(mask).mean(axis=2)/255 # shape(244,244)
y_true = y_true.reshape((1,224,224,1))

In [99]:
H = K.eval(NLL(K.variable(y_true), K.variable(y_true)))
J = K.eval(jaccard(K.variable(y_true), K.variable(y_true)))
K.eval(H - K.log(J))

In [104]:
K.eval(customLoss(K.variable(y_true), K.variable(y_true)))

0.30866325